### Let's try finding Venus' ephemerides in the ICRF coordinate system.

Small test here:

In [29]:
from skyfield.api import load
from skyfield import units
from astropy.time import Time

# Create a timescale and initialise beginning and end time.
ts = load.timescale()
t0 = Time('2011-08-18T10:00:00',scale ='utc')
t0= ts.from_astropy(t0)
print(t0.utc_jpl())
t1= ts.utc(2013)

# Load the JPL ephemeris DE421 (covers 1900-2050).
planets = load('de421.bsp')
venus = planets['venus']
sun = planets['sun']

# print(planets)
# print(venus)


A.D. 2011-Aug-18 10:00:00.0000 UTC


In [30]:
# The position of Venus, viewed from BCRS
venus_pos = venus.at(t0)
ra, dec, distance = venus_pos.radec() 

print(ra)
print(dec)
print(distance)
print(venus_pos)


09h 59m 15.86s
+15deg 40' 26.7"
0.72137 au
<Barycentric BCRS position and velocity at date t center=0 target=299>


In [31]:
# Let's convert this in degrees

import astropy.units as u

dec = dec.to(u.deg)
ra = ra.to(u.deg)

print("Position of Venus on the 18th august 2011 at 10am : ",dec, ra)

Position of Venus on the 18th august 2011 at 10am :  15.674088161227528 deg 149.81610096113602 deg


## Let's try the real thing now
Ok, now that we know how to compute ephemerides of Venus in degrees in the ICRF, let's try to send a job to ODA to 
find an image of venus with IBIS. What's INTEGRAL center of coordinate system?-> we'll work with astropy to avoid
confusion between date format and coordinate systems.

We know that IBIS has quite a large FoV so that serendipitous observations of Venus is possible. 
The goal now is to try find a moment when IBIS was observing a portion of the sky where Venus was.

Fully-coded IBIS FoV: 8.3°x8°. Down to zero: 29.1°x29.4°. What's the portion of the sky that this covers ? 
Whole sky is about 41'000 $deg^2$ so the FoV covers between 66.4 and 855.5 square degrees. 
The last number is quite elevated. Chances to find Venus are actually high I think.

What's Venus's apparent size in the sky on average ? Do we want to compute a probability of finding it ?
As in the 2001 paper, what could be interesting would be to find when Venus was close to its maximum elongation so that we have less 
pollution from the Sun.


## Finding Venus' max elongation.

In [80]:
from skyfield.api import load
from skyfield.framelib import ecliptic_frame
from skyfield.searchlib import find_maxima
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord

ts = load.timescale()
t0 = ts.from_astropy(Time('2006-01-01T00:00:00',scale ='utc'))
t1 = ts.from_astropy(Time('2019-12-31T00:00:00',scale ='utc'))

eph = load('de421.bsp')
sun, earth, venus = eph['sun'], eph['earth'], eph['venus']

def elongation_at(t):
    e = earth.at(t)
    s = e.observe(sun).apparent()
    v = e.observe(venus).apparent()
    return s.separation_from(v).degrees

elongation_at.step_days = 15.0

times, elongations = find_maxima(t0, t1, elongation_at)

for t, elongation_degrees in zip(times, elongations):
    e = earth.at(t)
    _, slon, _ = e.observe(sun).apparent().frame_latlon(ecliptic_frame)
    _, vlon, _ = e.observe(venus).apparent().frame_latlon(ecliptic_frame)
    is_east = (vlon.degrees - slon.degrees) % 360.0 < 180.0
    direction = 'east' if is_east else 'west'
    print('{}  {:4.1f}° {} elongation'.format(
        t.utc_strftime(), elongation_degrees, direction))

2006-03-25 06:44:51 UTC  46.5° west elongation
2007-06-09 02:45:09 UTC  45.4° east elongation
2007-10-28 15:05:20 UTC  46.5° west elongation
2009-01-14 21:23:37 UTC  47.1° east elongation
2009-06-05 20:50:55 UTC  45.9° west elongation
2010-08-20 03:48:19 UTC  46.0° east elongation
2011-01-08 16:01:38 UTC  47.0° west elongation
2012-03-27 07:44:19 UTC  46.0° east elongation
2012-08-15 09:06:35 UTC  45.8° west elongation
2013-11-01 07:58:41 UTC  47.1° east elongation
2014-03-22 19:31:28 UTC  46.6° west elongation
2015-06-06 18:29:24 UTC  45.4° east elongation
2015-10-26 07:11:11 UTC  46.4° west elongation
2017-01-12 13:18:27 UTC  47.1° east elongation
2017-06-03 12:29:48 UTC  45.9° west elongation
2018-08-17 17:31:17 UTC  45.9° east elongation
2019-01-06 04:53:35 UTC  47.0° west elongation


Let's compute the positions for all these 17 maximum elongations.

In [81]:
venus_pos=venus.at(times)
ra, dec, distance = venus_pos.radec() 
dec = dec.to(u.deg)
ra = ra.to(u.deg)
is_east=-1
for dec_obj, ra_obj,t,elongation_degrees in zip(dec, ra,times,elongations):
    is_east=(is_east+1)%2
    direction = 'east' if is_east else 'west'
    print('Position of Venus on the {} for a {:4.1f}° {} elongation: RA: {:4.5f}, Dec: {:4.5f}'.format(t.utc_strftime(), elongation_degrees, direction, ra_obj,dec_obj))

Position of Venus on the 2006-03-25 06:44:51 UTC for a 46.5° west elongation: RA: 225.37855 deg, Dec: -15.41292 deg
Position of Venus on the 2007-06-09 02:45:09 UTC for a 45.4° east elongation: RA: 211.90738 deg, Dec: -10.43616 deg
Position of Venus on the 2007-10-28 15:05:20 UTC for a 46.5° west elongation: RA: 77.90187 deg, Dec: 23.09685 deg
Position of Venus on the 2009-01-14 21:23:37 UTC for a 47.1° east elongation: RA: 69.33370 deg, Dec: 21.71622 deg
Position of Venus on the 2009-06-05 20:50:55 UTC for a 45.9° west elongation: RA: 301.41201 deg, Dec: -22.65306 deg
Position of Venus on the 2010-08-20 03:48:19 UTC for a 46.0° east elongation: RA: 285.53305 deg, Dec: -24.26791 deg
Position of Venus on the 2011-01-08 16:01:38 UTC for a 47.0° west elongation: RA: 154.39273 deg, Dec: 14.09857 deg
Position of Venus on the 2012-03-27 07:44:19 UTC for a 46.0° east elongation: RA: 145.37028 deg, Dec: 17.09323 deg
Position of Venus on the 2012-08-15 09:06:35 UTC for a 45.8° west elongation: 

Now we try to find scw for these position and dates. I'll try with the web interface first. What should I put in source name ?
We'll first try with 2012-03-27 07:44:19 UTC and position RA: 145.37028 deg, Dec: 17.09323 deg.

In [82]:
import astroquery.heasarc
from astroquery.heasarc import Heasarc, Conf
import json
import os
import shutil
import random
from astropy.io import fits
from scipy import stats

Heasarc = astroquery.heasarc.Heasarc()
Conf.server.set('https://www.isdc.unige.ch/browse/w3query.pl')
table = Heasarc.query_mission_list()
table.pprint(max_width=120)

def get_scw_list(ra_obj, dec_obj,radius,start_date,end_date ):
    R = Heasarc.query_region(
            position = SkyCoord(ra_obj, dec_obj, unit='deg'),
            radius = f"{radius} deg",
            mission = 'intscw',
            time = start_date + " .. " + end_date,
            good_isgri = ">1000",
        )

    R.sort('SCW_ID')

    return R['SCW_ID'], R['SCW_VER']

assert astroquery.__version__ >= '0.4.2.dev6611'

# it means it's our fork
assert 'isdc' in astroquery.heasarc.Conf.server.cfgtype

radius = 10.
ra_test = ra[7]
dec_test = dec[7]
t_test = times[7]
t_in = times[7]
print(ra_test, dec_test,t_test.utc_strftime(),t_test.utc_iso(),times[7])

   Mission            Table                         Table Description               
------------- ---------------------- -----------------------------------------------
CTASST1M-REV1     cta_sst1m_rev1_run                                             Run
    FACT-REV1          fact_rev1_run                                             Run
INTEGRAL-REV3     integral_rev3_prop                                       Proposals
INTEGRAL-REV3 integral_rev3_prop_obs Proposal Information and Observation Parameters
INTEGRAL-REV3      integral_rev3_scw                       SCW - Science Window Data
145.3702781997043 deg 17.09322558279773 deg 2012-03-27 07:44:19 UTC 2012-03-27T07:44:19Z <Time tt=2456013.823212227>


In [83]:
scwlist,version = get_scw_list(ra_test,dec_test, radius, '2012-03-20T00:00:00', '2012-03-30T00:00:00')

c:\Users\kentb\miniconda3\envs\spacetech_env\Lib\site-packages\astroquery\utils\commons.py:181: InputWarning: Coordinate string is being interpreted as an ICRS coordinate provided in degrees.
  warnings.warn("Coordinate string is being interpreted as an "


IORegistryError: Format could not be identified based on the file name or contents, please provide a 'format' argument.
The available formats are:
           Format           Read Write Auto-identify Deprecated
--------------------------- ---- ----- ------------- ----------
                      ascii  Yes   Yes            No           
               ascii.aastex  Yes   Yes            No           
                ascii.basic  Yes   Yes            No           
                  ascii.cds  Yes    No            No           
     ascii.commented_header  Yes   Yes            No           
                  ascii.csv  Yes   Yes           Yes           
              ascii.daophot  Yes    No            No           
                 ascii.ecsv  Yes   Yes           Yes           
           ascii.fast_basic  Yes   Yes            No           
ascii.fast_commented_header  Yes   Yes            No           
             ascii.fast_csv  Yes   Yes            No           
       ascii.fast_no_